In [3]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
height, width, _ = frame.shape
print(f'shape {width}, {height}')

coords = []
drawing = False
roi = None
mouse_x = None
mouse_y = None
wzor = None


def click_and_crop(event, x, y, flag, image):
    global coords, drawing, roi, mouse_x, mouse_y,wzor,wykryto_kontury
    mouse_x = x
    mouse_y = y
    if event == cv2.EVENT_LBUTTONDOWN:
        # user has clicked the mouse's left button
        drawing = True
        # save those starting coordinates
        coords = [(x, y)]
    elif event == cv2.EVENT_MOUSEMOVE:
    # user is moving the mouse within the window
        if drawing:
            # if we're in drawing mode, we'll draw a green rectangle
            # from the starting x,y coords to our current coords
            clone = frame.copy()
            #cv2.rectangle(frame, coords[0], (x, y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
    # user has released the mouse button, leave drawing mode
    # and crop the photo
        drawing = False
        # save our ending coordinates
        coords.append((x, y))
        if len(coords) == 2:
            # calculate the four corners of our region of interest
            if coords[0][0] > coords[1][1]:
                first = coords[1]
                second = coords [0]
                coords[0] = first
                coords[1] = second
            ty, by, tx, bx = coords[0][1], coords[1][1], coords[0][0], coords[1][0]
            # crop the image using array slicing
            wzor = frame[ty:by, tx:bx]
            height, width = wzor.shape[:2]
            if width > 0 and height > 0:
                wykryto_kontury = False
                # make sure roi has height/width to prevent imshow error
                # and show the cropped image in a new window
                cv2.namedWindow("wzor")
                cv2.imshow("wzor", wzor)
                



cv2.namedWindow('kamera')
cv2.setMouseCallback('kamera',click_and_crop)
wykryto_kontury = False;
                
while True:
    i = 0
    ret, frame = cap.read()
    copy_frame = frame.copy()
    if drawing:
        cv2.rectangle(copy_frame, coords[0], (mouse_x, mouse_y), (0, 255, 0), 2)
    
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur_frame = cv2.medianBlur(frame_gray,5)
    edges_frame = cv2.Canny(blur_frame,30,200)
    ret, frame_thresh = cv2.threshold(edges_frame,127,255,0)
    f_contours, hierarchy = cv2.findContours(frame_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if f_contours:
        cv2.drawContours(copy_frame, f_contours, -1,(0,255,0),3)
        pass
    
    if wzor is not None:
        if not wykryto_kontury:
            gray_wzor = cv2.cvtColor(wzor, cv2.COLOR_BGR2GRAY)
            blur_wzor = cv2.medianBlur(gray_wzor,5)
            edges_wzor = cv2.Canny(blur_wzor,30,200)
            ret, wzor_thresh = cv2.threshold(edges_wzor,127,255,0)
            w_contours, hierarchy = cv2.findContours(wzor_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_contours = sorted(w_contours, key=cv2.contourArea,reverse=True)

            if len(w_contours) > 10:
                w_contours = w_contours[:10]
            cv2.drawContours(wzor, w_contours, -1,(0,255,0),3)
            cv2.imshow("wzor", wzor)
            wykryto_kontury = True
        
        
        for c in f_contours:
            for d in w_contours:
                match = cv2.matchShapes(d,c,1,0.0)
                #print(match)

                if match < 0.2:
                    closet_contour = c
                    cv2.drawContours(copy_frame, [closet_contour], -1,(255,0,0),3)
                    i+=1
                    #print(i)
                    
                else:
                    #closet_contour = []
                    pass
        
    cv2.putText(copy_frame,f'zgodne kontury {i}', (30,30), cv2.FONT_HERSHEY_COMPLEX,1,(100,170,0),2)
    cv2.imshow('kamera',copy_frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
    
    
    
cap.release()
cv2.destroyAllWindows()

shape 640, 480
